In [ ]:
!pip install simpletransformers

import numpy as np
import pandas as pd

import torch
import torch.nn as nn

from simpletransformers.ner import NERModel
from simpletransformers.classification import ClassificationModel


# Load Dataset

In [ ]:
from google.colab import drive
drive.mount("/content/drive")
cols = ["text","label"]
df = pd.read_csv(
    "/content/drive/MyDrive/Bachelor/df_DK.csv",
    header=0,
    engine="python",
    encoding="UTF-8"
)
df = df[["Transcription", "Backchannel", ".folds", "index"]]
df.columns = ["text", "label", "fold", "num"]

Mounted at /content/drive


In [ ]:
device = torch.device("cuda")

In [ ]:
n_labels = len(df['label'].unique())

# initialize the model
sent_model = ClassificationModel('bert', "/content/drive/My Drive/danish_bert_uncased_v2/",
                                 num_labels=n_labels, use_cuda=True,
                                 args={'reprocess_input_data': True, 'overwrite_output_dir': True,
                                       "num_train_epochs": 2, 'max_seq_length': 32})

Some weights of the model checkpoint at /content/drive/My Drive/danish_bert_uncased_v2/ were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceCl

In [ ]:
prs = [i for i in df['fold'].unique()]

In [ ]:
prs=[3, 1, 2,4,8, 6, 7, 5, 10, 9]

In [ ]:
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [ ]:
for i in prs:
  testdata = df[df["fold"] == i]
  traindata = df[df["fold"]!=i]
  sent_model.train_model(traindata)

  result, model_outputs, wrong_predictions = sent_model.eval_model(testdata, acc=sklearn.metrics.accuracy_score)

  testdata.index = [i for i in range(len(testdata.text))]

  predictions, raw_outputs = sent_model.predict(testdata.text)
  df_res = pd.DataFrame(list(zip(testdata.num, predictions)), 
               columns =['index', 'bert_preds'])
  filename = [str(i), "fold", "df.csv"]
  df_res.to_csv("_".join(filename))
  